In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm
tqdm.pandas()

## Data collection

In [ ]:
df = pd.read_csv('trac2_eng_dev.csv')
df2 = pd.read_csv('trac2_eng_train.csv')
final = pd.concat([df,df2])
final.to_csv('TRAC_raw.csv',index=False, encoding='utf-8-sig')

## Data preprocess

In [ ]:
df = pd.read_csv('TRAC_raw.csv')

def clean_text(text):
    text = " ".join(filter(lambda x: x[0]!= '@' , text.split()))
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r'MENTION', ' ', text)
    text = re.sub(r'RT', ' ', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'(?<!\.)\.{3,}(?!\.)', '', text)  
    text = re.sub(r'[^\w\s]', '', text) 
    text = re.sub(r'\b\d+(\.\d+)?', '', text)  
    text = re.sub(r'\b\d+(\.\d+)?', '', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'_', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'http', '', text)
    text = re.sub(r'https', '', text)
    text = re.sub(r'nt', '', text)
    text = re.sub(r'mkr', '', text)
    text = re.sub(r've', '', text)
    text = re.sub(r'don', '', text)
    text = re.sub(r'kat', '', text)
    text = re.sub(r'kabir', '', text)
    text = re.sub(r'singh', '', text)
    text = re.sub(r'bhai', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = text.split()
    text = " ".join(text)

    return text

In [ ]:
df['prepro'] = df['Text'].progress_apply(clean_text)
df['prepro'] = df['prepro'].apply(lambda x: str(x) if pd.notnull(x) else '')
df = df[df['prepro'].apply(lambda x: len(x.split()) > 3)]
df['encoding'] = df['Sub-task A'].apply(lambda x: 2 if x == 'OAG' else (1 if x == 'CAG' else (0 if x == 'NAG' else -1)))
df = df.drop(columns=['ID', 'Text', 'Sub-task A', 'Sub-task B'])

df.to_csv('TRAC_input.csv', index=False, encoding='utf-8-sig')